# Playlist Generation

### Goal: 

Step 1: From a seed song find all songs from all related artists

### Import Libraries and create spotify credentials manager

In [4]:
#importing module spotipy to make api calls to spotify
import spotipy
# module to extract song from url
import urllib
# OAuth Credenials
from spotipy.oauth2 import SpotifyClientCredentials
from multiprocessing import Process, Queue
import pandas as pd 
import numpy as np
from sklearn import datasets, linear_model
#from sklearn.model_selection import cross_val_predict
from sklearn import linear_model
import matplotlib.pyplot as plt

client_credentials_manager = SpotifyClientCredentials(client_id='4a1a457d14c0445a96e7021a5523905b', client_secret='45b186d601544fdcbb4fe07c7b23cce5')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


### Read in csv that has the track ids from all Spotify playlists

In [5]:
track_info = pd.read_csv("kmeans_output.csv")

In [6]:
track_ids = list(track_info['id'])

In [7]:
len(track_ids)
print(track_ids[:15])

['1tPGSiwtvmMVAqmniJ7oHp', '1tpShM80HMT9vKjRiA3EzW', '1TPUbLdzpfgF7wrPEXIAAB', '1tQGRq2WOBXjL3JWdWMONg', '1TqmZVIyeD4aN4Wa0wbnfy', '1TquBahz02HCaxj8jy5tTo', '1Tr4K5MU5XYE44umXGDndd', '1tRlCCMVLSVQrrN9PPgtf0', '1TScJLsDWzDVBFpCU5vJDo', '1TSjS0q24cPW2S4pYj2wEK', '1Tt4sE4pXi57mTD1GCzsqm', '1TtdHZ4KvN56LCt9evBUaL', '1TTiZwoorWMR4KmaI0irjU', '1TtJJ7YP6JCQ3aH8PQpreD', '1TTOAw90zzNWKnUgzzcvZW']


In [8]:
track_result = sp.tracks(["1tpShM80HMT9vKjRiA3EzW"])

### Input a seed track

In [9]:
seed_id = "7qiZfU4dY1lWllzX7mPBI3"
#seed_id = track_ids[3]
track_info = sp.tracks([seed_id])
name = track_info['tracks'][0]['name']
seed_preview_url = track_info['tracks'][0]['preview_url']
print(seed_id,name)

7qiZfU4dY1lWllzX7mPBI3 Shape of You


In [10]:
track_info['tracks'][0]['artists']

[{u'external_urls': {u'spotify': u'https://open.spotify.com/artist/6eUKZXaKkcviH0Ku9w2n3V'},
  u'href': u'https://api.spotify.com/v1/artists/6eUKZXaKkcviH0Ku9w2n3V',
  u'id': u'6eUKZXaKkcviH0Ku9w2n3V',
  u'name': u'Ed Sheeran',
  u'type': u'artist',
  u'uri': u'spotify:artist:6eUKZXaKkcviH0Ku9w2n3V'}]

### Get the track's artists

In [11]:
# Get artist
artist_names = []
artist_ids = []
for each in track_info['tracks'][0]['artists']:
    artist_names.append(each['name'])
    artist_ids.append(each['id'])
print(artist_names)
print(artist_ids)

[u'Ed Sheeran']
[u'6eUKZXaKkcviH0Ku9w2n3V']


### Get related artists

In [12]:
all_related_artists_names = []
all_related_artists_ids = []

for each in artist_ids:
    related = sp.artist_related_artists(each)
    for related_artist in related['artists']:
        all_related_artists_names.append(related_artist['name'])
        all_related_artists_ids.append(related_artist['id'])

In [13]:
all_related_artists_ids = np.array(all_related_artists_ids)

In [14]:
len(all_related_artists_ids)

20

In [15]:
all_related_artists_ids_unique = np.unique(all_related_artists_ids)
len(all_related_artists_ids_unique)

20

### Bucket #1: Top tracks from the related artists

In [16]:
#top_tracks_one_artist

In [17]:
top_tracks_related_artists = []
for each in all_related_artists_ids_unique:    
    top_tracks_one_artist = sp.artist_top_tracks(each)

    for each in top_tracks_one_artist['tracks']:
        song = each['id']
        top_tracks_related_artists.append(song)
        


In [18]:
len(top_tracks_related_artists)

200

### Bucket #2: Songs from related artists that are not in top tracks but are similar

In [19]:
all_related_albums = []
for each in all_related_artists_ids_unique:    
    albums = sp.artist_albums(each, album_type=None, country=None, limit=50, offset=0)
    for each in albums['items']:
        album_id = each['id']
        all_related_albums.append(album_id)

In [20]:
len(all_related_albums)

1000

In [21]:
all_related_albums[0:10]

[u'5nhoVKDcCXvi3AruUFJWnv',
 u'20Ol6zZ0nLlc5EGTH1zA0j',
 u'3XpjGUy3t7fQ464SweCuZj',
 u'3Yps9hZIyZHjg5g4H2xoyW',
 u'2P69jY1Uwy9xqyAQuNA1St',
 u'4A43DyDoAVa1Fb8pq6Yejl',
 u'7DBwbOlvvFfHtJ9Fyq7sv0',
 u'2Dw4fYqDQnxsgoXDdMbqh3',
 u'3KVeczHxWg5YFKb0gS62f2',
 u'7epiarahRkAHJbwASl4hA5']

In [22]:
sp.album_tracks(all_related_albums[1], limit=50, offset=0)

all_related_songs = []
all_related_preview_urls = []
for index, each_album in enumerate(all_related_albums):
    album_tracks = sp.album_tracks(each_album, limit=50, offset=0)
    if index%50==0: print(index)
    for each in album_tracks['items']:
        all_related_songs.append(each['id'])
        all_related_preview_urls.append(each['preview_url'])

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950


### Total number of related songs for a seed track

In [23]:
len(all_related_songs)

12034

In [24]:
all_related_songs = np.array(all_related_songs)
all_related_songs = list(np.unique(all_related_songs))

In [25]:
combined = [seed_id]
combined.extend(all_related_songs)
all_related_songs = combined

In [26]:
seed_id

'7qiZfU4dY1lWllzX7mPBI3'

In [27]:
len(all_related_songs)

10656

In [28]:
all_related_songs[0]

'7qiZfU4dY1lWllzX7mPBI3'

### Test to make sure the songs sound like they're from related artists

In [29]:
from IPython.core.display import HTML
link = '<a href=' + str(seed_preview_url) + '>preview input song</a>'
HTML(link)


In [30]:
link = '<a href=' + str(all_related_preview_urls[10]) + '>preview related song</a>'
HTML(link)

In [31]:
audio_features = sp.audio_features(tracks=seed_id)[0]

### Get all Spotify features for input song and related songs

In [32]:
import math

In [39]:
def get_spotify_features(track_ids,numLoops=None):
    track_ids_all= []
    popularity = []
    preview_urls_all =[]
    artists_all = []
    names_all = []
    track_numbers_all = []
    num_markets = []
    
    # set parameters
    offset=0
    limit=50
    if numLoops==None: 
        numLoops=int(math.floor(len(track_ids)/50)+1)
    aud_features = np.empty((limit*numLoops, 13))
    for loopIndex in range(numLoops):
        start = offset + limit*loopIndex # start 
        tracks = track_ids[start:start+limit]
        try:
            tracks_list = sp.tracks(tracks)
            features = sp.audio_features(tracks=tracks)
        except:
            print("Couldn't get data", index)
            continue
        for index, track in enumerate(tracks_list['tracks']):
            
            try:
                if (start+index)%1000==0: 
                    print(index+1+start)
                preview_urls_all.append(track['preview_url'])
                track_ids_all.append(track['id'])
                popularity.append(track['popularity'])
                names_all.append(track['name'])
                artists_all.append(track['artists'][0]['name'])
                track_numbers_all.append(track['track_number'])
                num_markets.append(track['available_markets'])
                songData = features[index]
                aud_features[index+start][0] = songData['acousticness']
                aud_features[index+start][1] = songData['danceability']
                aud_features[index+start][2] = songData['energy']
                aud_features[index+start][3] = songData['instrumentalness']
                aud_features[index+start][4] = songData['key']
                aud_features[index+start][5] = songData['liveness']
                aud_features[index+start][6] = songData['loudness']
                aud_features[index+start][7] = songData['mode']
                aud_features[index+start][8] = songData['tempo']
                aud_features[index+start][9] = songData['time_signature']
                aud_features[index+start][10] = songData['valence']
                aud_features[index+start][11] = songData['duration_ms']
                aud_features[index+start][12] = songData['speechiness']
                 
            except:
                print("Some fields not available", index)

    aud_features = aud_features[:len(names_all),]
    track_data = pd.DataFrame({
    'preview_url': preview_urls_all,
    'id': track_ids_all,
    'popularity': popularity,
    'artists': artists_all,
    'names': names_all,
    'acousticness': aud_features[:,0],
    'danceability': aud_features[:,1],
    'energy': aud_features[:,2],
    'instrumentalness': aud_features[:,3],
    'key': aud_features[:,4],
    'liveness': aud_features[:,5],
    'loudness': aud_features[:,6],
    'mode': aud_features[:,7],
    'tempo': aud_features[:,8],
    'time_signature': aud_features[:,9],
    'valence': aud_features[:,10],
    'duration': aud_features[:,11],
    'speechiness': aud_features[:,12],
    'available_markets': num_markets
    
   
    })
    #q.put(track_data)
    print(len(num_markets), len(popularity), aud_features.shape)
    #name = multiprocessing.current_process().name
    print('Exiting')
    return track_data


In [40]:
features_df = get_spotify_features(track_ids=all_related_songs)

1
1001
2001
3001
Some fields not available 27
4001
5001
6001
7001
8001
9001
10001
10656 10656 (10656, 13)
Exiting


In [41]:
features_df.ix[10650:,]

,acousticness,artists,available_markets,danceability,duration,energy,id,instrumentalness,key,liveness,loudness,mode,names,popularity,preview_url,speechiness,tempo,time_signature,valence
10650,0.5780,Zara Larsson,"[AD, AR, AT, AU, BE, BG, BO, BR, CA, CH, CL, C...",0.699,189057.0,0.435,7zqLBFKCBkk5IfbgKgH4VZ,0.000000,1.0,0.0884,-6.891,1.0,Sexual - Recorded at Spotify Studios NYC,67,https://p.scdn.co/mp3-preview/394ef5e2e904e50e...,0.1110,110.288,4.0,0.462
10651,0.0123,The Vamps,"[AD, AR, AT, AU, BE, BG, BO, BR, CH, CL, CO, C...",NaN,212904.0,0.848,7zsxEwa4azEu5wRWdIuiQV,0.000000,1.0,0.4110,-3.714,1.0,All Night - Alex Adair Remix,59,https://p.scdn.co/mp3-preview/6447f8cabd59bee9...,NaN,0.000,NaN,NaN
10652,0.4120,Anastacia,"[GB, IE]",0.759,243080.0,0.747,7zuUlMu7JvptWNOE7It5Zw,0.000000,10.0,0.1030,-4.623,0.0,I'm Outta Love,22,https://p.scdn.co/mp3-preview/a8acea1f48207222...,0.0645,119.401,4.0,0.636
10653,0.2330,Emeli Sandé,"[ID, MY, TR]",0.564,229720.0,0.451,7zwmbO1PqRPCg0JY3Bf4m4,0.068400,2.0,0.1180,-10.661,1.0,I’d Rather Not,0,https://p.scdn.co/mp3-preview/406e0b5bed9c3054...,0.0415,146.145,4.0,0.170
10654,0.9700,Britney Spears,"[AD, AR, AT, BE, BG, BO, BR, CA, CH, CL, CO, C...",0.570,221160.0,0.272,7zy36kJCyUohmpGPKrdvzu,0.000164,3.0,0.1190,-13.232,1.0,Everytime,22,https://p.scdn.co/mp3-preview/0e1bdb7c05b02585...,0.0298,109.522,4.0,0.135
10655,0.7800,Zara Larsson,[SE],0.597,155520.0,0.627,7zzSgIOkexLBQA98Wj57dc,0.000000,3.0,0.1650,-4.912,0.0,Bad Boys - Today Remix,28,https://p.scdn.co/mp3-preview/b277534925d4e356...,0.0348,113.050,4.0,0.290


In [42]:
get_spotify_features(track_ids=[all_related_songs[0]])

1
1 1 (1, 13)
Exiting


,acousticness,artists,available_markets,danceability,duration,energy,id,instrumentalness,key,liveness,loudness,mode,names,popularity,preview_url,speechiness,tempo,time_signature,valence
0,0.581,Ed Sheeran,"[AD, AR, AT, AU, BE, BG, BO, BR, CA, CH, CL, C...",0.825,233713.0,0.652,7qiZfU4dY1lWllzX7mPBI3,0.0,1.0,0.0931,-3.183,0.0,Shape of You,99,https://p.scdn.co/mp3-preview/84462d8e1e4d0f9e...,0.0802,95.977,4.0,0.933


## Find most similar songs acoustically

### Calculate the distance between the input song and related songs

In [43]:
from scipy.spatial.distance import cityblock

In [44]:
distance_features = features_df[['acousticness', 'danceability', 
                                 'duration', 'energy', 'instrumentalness', 'liveness', 
                                 'loudness', 'speechiness', 'tempo', 'valence']]

In [45]:
distance_features.head()

,acousticness,danceability,duration,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence
0,0.58100,0.825,233713.0,0.652,0.00000,0.0931,-3.183,0.0802,95.977,0.933
1,0.13100,0.748,188491.0,0.627,0.00000,0.0852,-6.029,0.0644,120.963,0.513
2,0.44700,0.597,195427.0,0.543,0.00000,0.2750,-10.280,0.3100,179.880,0.784
3,0.09170,0.633,217040.0,0.711,0.00178,0.4770,-7.508,0.1090,128.012,0.375
4,0.00883,0.702,268947.0,0.824,0.00000,0.0766,-4.724,0.0343,104.004,0.785


In [46]:
seed_features = distance_features.ix[0,]
related_features = distance_features.ix[1:,]

In [47]:
len(related_features)

10655

In [48]:
sample_size = len(related_features)
distance_vector = np.empty((sample_size))
for index,row in related_features.iterrows():
    if index>sample_size-1: break
    #print(row)
    distance_vector[index]=cityblock(seed_features, row)

In [49]:
distance_vector.shape

(10655,)

In [50]:
distance_vector[10015]

199256.65419999999

### Sort the distance output and find the 100 most acoustically similar songs

In [51]:
min_indices = distance_vector.argsort()[0:100:]

In [52]:
min_indices

array([    0,  9284,  1690,  3814,  7181,  9329,   830,   632,  7758,
        7518,   763,  4910,  5590,  6539,  4226,  1925,  7666,  3235,
         985,   525, 10343,  7894,  8734,  3036,  4724,  6898,  7213,
        3276,  8247,  4813,  3312,  2514, 10507,  8439,  5807,  6044,
        8623,  6914,  5707,  6282,  6722,  7585,  6491,  8771,  7093,
        4344,  8775,  8593,  2548,  2925,  6511,   152,   499,  1705,
        4955,  2878,  3695,  2664,  9269,  5701,  7820,    89,  8157,
        5176,  5952,   595,  9842,  2151,    31,  4621, 10356,  5678,
        5114,  2594,  4894,  8802,  4006,   904, 10341,  1698,  3907,
        5090,  3297,  4274,  7303,  5856,  5999,  7497,  2744,  6169,
         591,   392,  4048,  6019,  6723,  5118,  3267,  4843,  1349,  8627])

In [53]:
top_similarity_index = min_indices[2]
top_similarity_index

1690

In [54]:
related_all_features_with_seed = features_df.ix[:]
seed_all_features = features_df.ix[0,]

In [55]:
#features_df.head()

In [56]:
link = '<a href=' + str(seed_all_features['preview_url']) + '>preview seed song</a>'
HTML(link)

In [57]:
links = []
for each in range(10):
    links.append('<a href=' + str(related_all_features_with_seed['preview_url'][min_indices[each]]) + '>preview related song</a>')
HTML(links[3])

In [58]:
features_df.to_csv('related_songs.csv', encoding='utf-8')

# Optimization

## 1. Test Omar's prediction function

In [59]:
## Popularity Prediction for New Playlist

#libraries

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.cross_validation import train_test_split
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
import sys
from sklearn.metrics import mean_squared_error
from math import sqrt

# Step 1 - train the random forest

api_data = pd.read_csv('playlist_data_with_sequencing_4.8.17.csv')
api_data = api_data.dropna()
del api_data['Unnamed: 0']
del api_data['names']
del api_data['playlist_id']
api_data.head()

a = pd.qcut(api_data["followers"], 5,labels=range(1,6))
api_data['followers'] = np.array(a)

y = api_data['followers']
X = api_data.drop(['followers'],axis=1)
X = X.drop(['featured'],axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)


rfc = RFC(n_estimators = 100, max_depth = 29)
rfc.fit(X_train, y_train)

client_credentials_manager = SpotifyClientCredentials(client_id='df846cfd28e745178054587b3484f91c',client_secret='e3d39fc92a954e028ff1490288f3fe5c')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


In [60]:
def getTrack(track_id):
    #print(track_id)
    listing = features_df[features_df['id']==track_id]
    popularity = listing['popularity'].values[0]
    num_markets = listing['available_markets'].values[0]
    return {"popularity": popularity,
           'available_markets': num_markets}

In [61]:
def getAudioFeatures(list_of_ids):
    listing = features_df[features_df['id'].isin(list_of_ids)]
    return listing[['acousticness', 'danceability', 
                                 'energy', 'instrumentalness', 'key','liveness', 
                                 'loudness', 'mode','tempo','time_signature','valence','duration']].values

In [62]:
getAudioFeatures([seed_id, "7zwmbO1PqRPCg0JY3Bf4m4"])

array([[  5.81000000e-01,   8.25000000e-01,   6.52000000e-01,
          0.00000000e+00,   1.00000000e+00,   9.31000000e-02,
         -3.18300000e+00,   0.00000000e+00,   9.59770000e+01,
          4.00000000e+00,   9.33000000e-01,   2.33713000e+05],
       [  2.33000000e-01,   5.64000000e-01,   4.51000000e-01,
          6.84000000e-02,   2.00000000e+00,   1.18000000e-01,
         -1.06610000e+01,   1.00000000e+00,   1.46145000e+02,
          4.00000000e+00,   1.70000000e-01,   2.29720000e+05]])

In [63]:

def chunks(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0
    
    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
        
    return out

def gen_row(list_of_tracks):
            
    parts = chunks(list_of_tracks,4)
                    
    p1 = parts[0]
    p2 = parts[1]
    p3 = parts[2]
    p4 = parts[3]
    
    t_popularities = []
    t_num_markets = []
                
    for track in list_of_tracks:
        listing = getTrack(track)
        t_popularities.append(listing['popularity'])
        t_num_markets.append(len(listing['available_markets']))
                                        
    t_pop = np.mean(t_popularities)
    t_av_num_markets = np.mean(t_num_markets)

    #t_features = sp.audio_features(tracks = list_of_tracks)
    feat_len = len(list_of_tracks)
    #t_feature_matrix = np.empty((feat_len,12))
    t_feature_matrix = getAudioFeatures(list_of_tracks)
#     for ix, song in enumerate(t_features):
  
#         t_feature_matrix[ix][0] = song['acousticness']
#         t_feature_matrix[ix][1] = song['danceability']
#         t_feature_matrix[ix][2] = song['energy']
#         t_feature_matrix[ix][3] = song['instrumentalness']
#         t_feature_matrix[ix][4] = song['key']
#         t_feature_matrix[ix][5] = song['liveness']
#         t_feature_matrix[ix][6] = song['loudness']
#         t_feature_matrix[ix][7] = song['mode']
#         t_feature_matrix[ix][8] = song['tempo']
#         t_feature_matrix[ix][9] = song['time_signature']
#         t_feature_matrix[ix][10] = song['valence']
#         t_feature_matrix[ix][11] = song['duration_ms']
                            
    t_feature_median = np.percentile(t_feature_matrix,50,axis=0)
                    # Part 1
                    
    p1_popularities = []
    p1_num_markets = []
                    
    for track in p1:
        listing = getTrack(track)
        p1_popularities.append(listing['popularity'])
        p1_num_markets.append(len(listing['available_markets']))
    
    p1_pop = np.mean(p1_popularities)
    p1_av_num_markets = np.mean(p1_num_markets)
    
    #p1_features = sp.audio_features(tracks = p1)
    p1_len = len(p1)
    #p1_feature_matrix = np.empty((p1_len,12))
    p1_feature_matrix = getAudioFeatures(p1)
                    
#     for ix, song in enumerate(p1_features):
#         p1_feature_matrix[ix][0] = song['acousticness']
#         p1_feature_matrix[ix][1] = song['danceability']
#         p1_feature_matrix[ix][2] = song['energy']
#         p1_feature_matrix[ix][3] = song['instrumentalness']
#         p1_feature_matrix[ix][4] = song['key']
#         p1_feature_matrix[ix][5] = song['liveness']
#         p1_feature_matrix[ix][6] = song['loudness']
#         p1_feature_matrix[ix][7] = song['mode']
#         p1_feature_matrix[ix][8] = song['tempo']
#         p1_feature_matrix[ix][9] = song['time_signature']
#         p1_feature_matrix[ix][10] = song['valence']
#         p1_feature_matrix[ix][11] = song['duration_ms']
                        
    p1_feature_median = np.percentile(p1_feature_matrix,50,axis=0)
                    # Part 2
                    
    p2_popularities = []
    p2_num_markets = []
                    
    for track in p2:
        listing = getTrack(track)
        p2_popularities.append(listing['popularity'])
        p2_num_markets.append(len(listing['available_markets']))
    
    p2_pop = np.mean(p2_popularities)
    p2_av_num_markets = np.mean(p2_num_markets)
    
    #p2_features = sp.audio_features(tracks = p2)

    p2_len = len(p2)
    #p2_feature_matrix = np.empty((p2_len,12))
    p2_feature_matrix = getAudioFeatures(p2)
                 
#     for ix, song in enumerate(p2_features):
#         p2_feature_matrix[ix][0] = song['acousticness']
#         p2_feature_matrix[ix][1] = song['danceability']
#         p2_feature_matrix[ix][2] = song['energy']
#         p2_feature_matrix[ix][3] = song['instrumentalness']
#         p2_feature_matrix[ix][4] = song['key']
#         p2_feature_matrix[ix][5] = song['liveness']
#         p2_feature_matrix[ix][6] = song['loudness']
#         p2_feature_matrix[ix][7] = song['mode']
#         p2_feature_matrix[ix][8] = song['tempo']
#         p2_feature_matrix[ix][9] = song['time_signature']
#         p2_feature_matrix[ix][10] = song['valence']
#         p2_feature_matrix[ix][11] = song['duration_ms']
                        
    p2_feature_median = np.percentile(p2_feature_matrix,50,axis=0)
    
                        # Part 3
                    
    p3_popularities = []
    p3_num_markets = []
                  
    for track in p3:
        listing = getTrack(track)
        p3_popularities.append(listing['popularity'])
        p3_num_markets.append(len(listing['available_markets']))
    
    p3_pop = np.mean(p3_popularities)
    p3_av_num_markets = np.mean(p3_num_markets)
    
    #p3_features = sp.audio_features(tracks = p3)
    p3_len = len(p3)
    #p3_feature_matrix = np.empty((p3_len,12))
    p3_feature_matrix = getAudioFeatures(p3)
                
#     for ix, song in enumerate(p3_features):
#         p3_feature_matrix[ix][0] = song['acousticness']
#         p3_feature_matrix[ix][1] = song['danceability']
#         p3_feature_matrix[ix][2] = song['energy']
#         p3_feature_matrix[ix][3] = song['instrumentalness']
#         p3_feature_matrix[ix][4] = song['key']
#         p3_feature_matrix[ix][5] = song['liveness']
#         p3_feature_matrix[ix][6] = song['loudness']
#         p3_feature_matrix[ix][7] = song['mode']
#         p3_feature_matrix[ix][8] = song['tempo']
#         p3_feature_matrix[ix][9] = song['time_signature']
#         p3_feature_matrix[ix][10] = song['valence']
#         p3_feature_matrix[ix][11] = song['duration_ms']
                        
    p3_feature_median = np.percentile(p3_feature_matrix,50,axis=0)
    
                        # Part 4
                    
    p4_popularities = []
    p4_num_markets = []
                    
    for track in p4:
        listing = getTrack(track)
        p4_popularities.append(listing['popularity'])
        p4_num_markets.append(len(listing['available_markets']))
    
    p4_pop = np.mean(p4_popularities)
    p4_av_num_markets = np.mean(p4_num_markets)
    
    #p4_features = sp.audio_features(tracks = p4)
    p4_len = len(p4)
    #p4_feature_matrix = np.empty((p4_len,12))
    p4_feature_matrix = getAudioFeatures(p4)
             
#     for ix, song in enumerate(p4_features):
#         p4_feature_matrix[ix][0] = song['acousticness']
#         p4_feature_matrix[ix][1] = song['danceability']
#         p4_feature_matrix[ix][2] = song['energy']
#         p4_feature_matrix[ix][3] = song['instrumentalness']
#         p4_feature_matrix[ix][4] = song['key']
#         p4_feature_matrix[ix][5] = song['liveness']
#         p4_feature_matrix[ix][6] = song['loudness']
#         p4_feature_matrix[ix][7] = song['mode']
#         p4_feature_matrix[ix][8] = song['tempo']
#         p4_feature_matrix[ix][9] = song['time_signature']
#         p4_feature_matrix[ix][10] = song['valence']
#         p4_feature_matrix[ix][11] = song['duration_ms']
                        
    p4_feature_median = np.percentile(p4_feature_matrix,50,axis=0)
           
    playlist_data = pd.Series({
                    #'followers': followers,
                    #'names' : names,
                    #'playlist_id' : playlist_ids,
                    'total_tracks' : len(list_of_tracks),
                    #'featured' : featured,
                    'acousticness': p1_feature_median[0],
                    'danceability': p1_feature_median[1],
                    'energy': p1_feature_median[2],
                    'instrumentalness': p1_feature_median[3],
                    'key': p1_feature_median[4],
                    'liveness': p1_feature_median[5],
                    'loudness': p1_feature_median[6],
                    'mode': p1_feature_median[7],
                    'tempo': p1_feature_median[8],
                    'time_signature': p1_feature_median[9],
                    'valence': p1_feature_median[10],
                    'duration': p1_feature_median[11],
                    'popularity':np.mean(t_pop),
                    'mean_num_markets': np.mean(t_av_num_markets),
                    #'avg_years' : t_release_date,

                    'p1_acousticness': p1_feature_median[0],
                    'p1_danceability': p1_feature_median[1],
                    'p1_energy': p1_feature_median[2],
                    'p1_instrumentalness': p1_feature_median[3],
                    'p1_key': p1_feature_median[4],
                    'p1_liveness': p1_feature_median[5],
                    'p1_loudness': p1_feature_median[6],
                    'p1_mode': p1_feature_median[7],
                    'p1_tempo': p1_feature_median[8],
                    'p1_time_signature': p1_feature_median[9],
                    'p1_valence': p1_feature_median[10],
                    'p1_duration': p1_feature_median[11],
                    'p1_popularity': p1_pop,
                    'p1_mean_num_markets': p1_av_num_markets,
                    #'p1_avg_years': p1_release_date,

                    'p2_acousticness': p2_feature_median[0],
                    'p2_danceability': p2_feature_median[1],
                    'p2_energy': p2_feature_median[2],
                    'p2_instrumentalness': p2_feature_median[3],
                    'p2_key': p2_feature_median[4],
                    'p2_liveness': p2_feature_median[5],
                    'p2_loudness': p2_feature_median[6],
                    'p2_mode': p2_feature_median[7],
                    'p2_tempo': p2_feature_median[8],
                    'p2_time_signature': p2_feature_median[9],
                    'p2_valence': p2_feature_median[10],
                    'p2_duration': p2_feature_median[11],
                    'p2_popularity': p2_pop,
                    'p2_mean_num_markets': p2_av_num_markets,
                    #'p2_avg_years': p2_release_date,

                    'p3_acousticness': p3_feature_median[0],
                    'p3_danceability': p3_feature_median[1],
                    'p3_energy': p3_feature_median[2],
                    'p3_instrumentalness': p3_feature_median[3],
                    'p3_key': p3_feature_median[4],
                    'p3_liveness': p3_feature_median[5],
                    'p3_loudness': p3_feature_median[6],
                    'p3_mode': p3_feature_median[7],
                    'p3_tempo': p3_feature_median[8],
                    'p3_time_signature': p3_feature_median[9],
                    'p3_valence': p3_feature_median[10],
                    'p3_duration': p3_feature_median[11],
                    'p3_popularity': p3_pop,
                    'p3_mean_num_markets': p3_av_num_markets,
                    #'p3_avg_years': p3_release_date,

                    'p4_acousticness': p4_feature_median[0],
                    'p4_danceability': p4_feature_median[1],
                    'p4_energy': p4_feature_median[2],
                    'p4_instrumentalness': p4_feature_median[3],
                    'p4_key': p4_feature_median[4],
                    'p4_liveness': p4_feature_median[5],
                    'p4_loudness': p4_feature_median[6],
                    'p4_mode': p4_feature_median[7],
                    'p4_tempo': p4_feature_median[8],
                    'p4_time_signature': p4_feature_median[9],
                    'p4_valence': p4_feature_median[10],
                    'p4_duration': p4_feature_median[11],
                    'p4_popularity': p4_pop,
                    'p4_mean_num_markets': p4_av_num_markets
                    #'p4_avg_years': p4_release_date
        })
    return playlist_data

def gen_score(list_of_tracks):
    row = gen_row(list_of_tracks)
    list_of_proba = rfc.predict_proba(row.reshape(1,-1))[0]
    #print(row)
    #list_of_proba = rfc.predict(row.reshape(1,-1))[0]
    return list_of_proba

In [64]:
all_related_songs[1:5]

[u'000xQL6tZNLJzIrtIgxqSl',
 u'001NMNauIqU8t70X48P8vX',
 u'005Ua0AZQDAtOpavz5kdpW',
 u'006dQ6824YZgSvIvBRhcJT']

In [65]:
result = gen_score(all_related_songs[10:20])
result

array([ 0.42,  0.2 ,  0.17,  0.19,  0.02])

## 2. Simple model

In [67]:
nona = features_df.dropna()
nona.shape
all_related_songs = nona['id'].values
all_related_songs.shape

(9013,)

In [68]:
# sample 5 different random samples and print the results
all_related_songs_np = np.array(all_related_songs[1:])
best_result = 0
best_playlist = []
for i in range(10):
    random_sample_10 = np.random.choice(all_related_songs_np, size=10)
    result = gen_score(random_sample_10)[4]
    if result>best_result:
        best_result=result
        best_playlist = random_sample_10
    print(result)
    
# select the best one


0.0
0.01
0.09
0.04
0.02
0.04
0.02
0.02
0.02
0.05


In [69]:
best_playlist

array([u'3KacmOuRkD3wwTMRBbWApo', u'1UvgTNJKD5Ym8lWXFVrDKW',
       u'35XDJTnw4ljwlSksWTuETX', u'5tbvdChR0xSR8JYuGsSpUY',
       u'1clmc70vYouNNjvLo4TcdY', u'6dq1fUiJe6BiBORZtb9TxZ',
       u'2PkTmV7ryvkUZf7eoEeAF3', u'0eoZZnFybiNwT3oir1dUrz',
       u'1zL880NLhgahwsnk8sNZ7C', u'7lJKCbYJrNPr9pIhB2hm3P'], dtype=object)

## 3. Simulated annealing

In [71]:
import simanneal
from __future__ import print_function
import math
import random
from simanneal import Annealer

In [72]:
class TravellingSalesmanProblem(Annealer):

    """Test annealer with a travelling salesman problem.
    """
    
    # pass extra data (the distance matrix) into the constructor
    def __init__(self, state, candidate_songs):
        self.candidate_songs = np.array(candidate_songs)
        #self.features = features
        super(TravellingSalesmanProblem, self).__init__(state)  # important! 

    def move(self):
        """Swaps two cities in the route."""
        a = random.randint(0, len(self.state) - 1)
        #b = random.randint(0, len(self.state) - 1)
        #self.state[a], self.state[b] = self.state[b], self.state[a]
        self.state[a] = np.random.choice(self.candidate_songs, size=1)[0]
        
    def energy(self): ## calculates the predicted number of followers
        """Calculates the predicted number of followers."""
        #print(self.state)
        result = gen_score(self.state)
        #print("Calculating energy: ", result)
        return 1.0-result[4]
        #return (1.0-(np.array([0,2.25,0.5,1,2])*result).mean())

In [73]:
if __name__ == '__main__':

    candidates = all_related_songs_np
    initial_playlist = best_playlist
    
    # initial state, a randomly-ordered itinerary
    init_state = list(initial_playlist)
    #random.shuffle(init_state)
    
    tsp = TravellingSalesmanProblem(init_state, candidates)
    tsp.updates = 100
    tsp.steps = 100
    tsp.Tmax = 24000
    tsp.Tmin = 2.5
    # since our state is just a list, slice is the fastest way to copy
    tsp.copy_strategy = "slice"  
    state, e = tsp.anneal()
    

    #while state[0] != 'New York City':
    #    state = state[1:] + state[:1]  # rotate NYC to start
    print("%s predicted probability:" % e)
    for city in state:
        print("\t", city)

 Temperature        Energy    Accept   Improve     Elapsed   Remaining



0.91 predicted probability:
	 3KacmOuRkD3wwTMRBbWApo
	 1UvgTNJKD5Ym8lWXFVrDKW
	 35XDJTnw4ljwlSksWTuETX
	 5tbvdChR0xSR8JYuGsSpUY
	 1clmc70vYouNNjvLo4TcdY
	 6dq1fUiJe6BiBORZtb9TxZ
	 2PkTmV7ryvkUZf7eoEeAF3
	 0eoZZnFybiNwT3oir1dUrz
	 1zL880NLhgahwsnk8sNZ7C
	 7lJKCbYJrNPr9pIhB2hm3P


In [99]:
#state

In [110]:
#auto_schedule = tsp.auto(minutes=0.1) 

In [75]:
from __future__ import print_function
import math
import random
from simanneal import Annealer


def distance(a, b):
    """Calculates distance between two latitude-longitude coordinates."""
    R = 3963  # radius of Earth (miles)
    lat1, lon1 = math.radians(a[0]), math.radians(a[1])
    lat2, lon2 = math.radians(b[0]), math.radians(b[1])
    return math.acos(math.sin(lat1) * math.sin(lat2) +
                     math.cos(lat1) * math.cos(lat2) * math.cos(lon1 - lon2)) * R


class TravellingSalesmanProblem(Annealer):

    """Test annealer with a travelling salesman problem.
    """
    
    # pass extra data (the distance matrix) into the constructor
    def __init__(self, state, distance_matrix):
        self.distance_matrix = distance_matrix
        super(TravellingSalesmanProblem, self).__init__(state)  # important! 

    def move(self):
        """Swaps two cities in the route."""
        a = random.randint(0, len(self.state) - 1)
        b = random.randint(0, len(self.state) - 1)
        self.state[a], self.state[b] = self.state[b], self.state[a]

    def energy(self):
        """Calculates the length of the route."""
        e = 0
        for i in range(len(self.state)):
            e += self.distance_matrix[self.state[i-1]][self.state[i]]
        return e



if __name__ == '__main__':

    # latitude and longitude for the twenty largest U.S. cities
    cities = {
        'New York City': (40.72, 74.00),
        'Los Angeles': (34.05, 118.25),
        'Chicago': (41.88, 87.63),
        'Houston': (29.77, 95.38),
        'Phoenix': (33.45, 112.07),
        'Philadelphia': (39.95, 75.17),
        'San Antonio': (29.53, 98.47),
        'Dallas': (32.78, 96.80),
        'San Diego': (32.78, 117.15),
        'San Jose': (37.30, 121.87),
        'Detroit': (42.33, 83.05),
        'San Francisco': (37.78, 122.42),
        'Jacksonville': (30.32, 81.70),
        'Indianapolis': (39.78, 86.15),
        'Austin': (30.27, 97.77),
        'Columbus': (39.98, 82.98),
        'Fort Worth': (32.75, 97.33),
        'Charlotte': (35.23, 80.85),
        'Memphis': (35.12, 89.97),
        'Baltimore': (39.28, 76.62)
    }

    # initial state, a randomly-ordered itinerary
    init_state = list(cities.keys())
    random.shuffle(init_state)

    # create a distance matrix
    distance_matrix = {}
    for ka, va in cities.items():
        distance_matrix[ka] = {}
        for kb, vb in cities.items():
            if kb == ka:
                distance_matrix[ka][kb] = 0.0
            else:
                distance_matrix[ka][kb] = distance(va, vb)

    tsp = TravellingSalesmanProblem(init_state, distance_matrix)
    # since our state is just a list, slice is the fastest way to copy
    tsp.copy_strategy = "slice"  
    state, e = tsp.anneal()

    while state[0] != 'New York City':
        state = state[1:] + state[:1]  # rotate NYC to start
    print("%i mile route:" % e)
    for city in state:
        print("\t", city)

 Temperature        Energy    Accept   Improve     Elapsed   Remaining



6882 mile route:
	 New York City
	 Philadelphia
	 Baltimore
	 Charlotte
	 Jacksonville
	 Houston
	 Austin
	 San Antonio
	 San Diego
	 Los Angeles
	 San Jose
	 San Francisco
	 Phoenix
	 Fort Worth
	 Dallas
	 Memphis
	 Indianapolis
	 Chicago
	 Detroit
	 Columbus
